In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")


# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature




from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as puti 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( puti )
importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
from myPythonTools.YAML import get_exp_list as Xl
importlib.reload(Xl)


In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}


In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('../YAML/configure_plotting.yaml', 'r') as file:
    cfgPlot = yaml.safe_load(file)


######################################################
season = 'ann' # cfgPlot['latlon']['season']
valData  = cfgPlot['latlon']['validation']['data']
valYears = cfgPlot['latlon']['validation']['years']
print( f"Season= {season}" )
print( valData , valYears )

# other ... 
fld='TAUX' #'PRECT'
fld='PRECT'
#fld='U10'

In [ ]:

exp_yaml,xl= Xl.Official()  #Utility()

with open(f'../YAML/{exp_yaml}', 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )
    A['Src']=Src
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    Al.append( A )
    print( f'{A.exp} Years {len(A.X.time) /12. } \n')


In [ ]:
Al[0].X

In [ ]:
########################################
# Make seasonal means

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')
########################################
    
    AAa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld=fld,return_time=True)
    A['AA_s']=AAa_s
    A['years']=years_a
    A['months']=months_a


In [ ]:
########################################
# Regrid to lat-lon if needed
########################################
method='C2'

for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    AAa_s = A['AA_s']
    
    if (Hkey == 'yx' ):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
        A['AA_s_yx'] = AAa_s

    elif (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        AAa_s_yx=RgF.Horz(xfld_Src=AAa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        A['AA_s_yx'] = AAa_s_yx

    A['lonx'] = lon_a
    A['latx'] = lat_a
    A['zlev'] = zlev_a
    A['plev'] = lev_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
"""
####
# Validation
####
mgrid = False

valYears='*'
Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
UUv_s_yx , zlev_v,lat_v,lon_v, years_Uv = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon'], Uval['years']

Tval = Val.data(fld='T',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
TTv_s_yx , zlev_v,lat_v,lon_v, years_Tv = Tval['aa'], Tval['lev'], Tval['lat'], Tval['lon'], Tval['years']
"""


In [ ]:

lev_sel = uti.find_nearest_plev_indices(plev=Al[0].zlev , target_levels=[ 1.0 ] )


In [ ]:
shape_AA = np.shape( Al[0].AA_s_yx )
print(len(shape_AA))


In [ ]:
print(fld)

In [ ]:
plt.contourf(Al[0].AA_s_yx)

In [ ]:
##############
plot_Z_km=1. #1.0
clev_prect=0.25*np.linspace(0,20,num=21)
#clev=np.linspace(0,1.0,num=21)

plot_Z_km=12. #1.0

clev_fld , dlev_fld, scale_field = puti.contour_intervals( fld )  #np.linspace(-0.3, 0.3 ,num=21) # 21

fields=[]
lons=[]
lats=[]
clevs=[]
titles=[]
cmaps=[]
insets=[]
control_diff = True #True
ipanel=0

for A in Al:
    years_a = A['years']
    if ( ( control_diff==True ) and ( ipanel>0) ):
        diff_fac = 1
        the_cmap = 'bwr_r'
        clev = 0.5*np.linspace(-1,1,num=21)
    else :
        diff_fac = 0
        the_cmap = 'Blues_r' #'gist_ncar'
        clev = clev_fld
    
    if ( len( np.shape( A.AA_s_yx ) ) == 3 ):
        kplot = uti.find_nearest_plev_indices(plev=A.zlev , target_levels=[ plot_Z_km ] )
        fields.append(  scale_field * A.AA_s_yx[kplot[0],:,:] )
        fldoo=f'{fld} (z={plot_Z_km:.1f}km) '
    elif ( len( np.shape( A.AA_s_yx ) ) == 2 ):
        fields.append(  scale_field * ( A.AA_s_yx[:,:] - diff_fac*Al[0].AA_s_yx[:,:]) )
        fldoo=f'{fld} '
        
    lons.append( A.lonx )
    lats.append( A.latx )
    clevs.append( clev )
    titles.append( f"{A.exp}: {fldoo} \n {season.upper()}  {years_a[0]}-{years_a[-1]}" )
    cmaps.append( the_cmap )

    if 'gw_parameters' in A:
        parameters=A['gw_parameters']
        # Parameters to display in the inset box
        selected_keys = ['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
        selected_params = {key: parameters[key] for key in selected_keys if key in parameters}
        # Format selected parameters into a string
        param_text = "\n".join([f"{key}: {value}" for key, value in selected_params.items()])
        inset={'text':param_text, 'x':0.65,'y':0.98}
        insets.append( inset )

    ipanel = ipanel+1

"""
##
# Now append validation
kplot = uti.find_nearest_plev_indices(plev=zlev_v , target_levels=[ plot_Z_km ] )
fields.append(  UUv_s_yx[kplot[0],:,:] )
lons.append( lon_v )
lats.append( lat_v )
clevs.append( clev )
titles.append( f"{fldoo} {season.upper()} ERA5 " )
cmaps.append( the_cmap )
"""

In [ ]:
####
MapProj = ccrs.PlateCarree(central_longitude=180.)

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , titles=titles, insets=insets, Projection=MapProj, Landmask=False )

In [ ]:
####
kplot = uti.find_nearest_plev_indices(plev=Al[0].zlev , target_levels=[ plot_Z_km ] )
print(Al[0].zlev[kplot])
print(Al[0].plev[kplot])


In [ ]:
#plt.plot( np.average( scale_field*Al[0].AA_s_yx , axis=1 ) )
plt.plot( Al[0].latx, np.average( scale_field*Al[0].AA_s_yx , axis=1 ) )
plt.plot( Al[0].latx, np.average( scale_field*Al[1].AA_s_yx, axis=1 ) )
plt.plot( Al[0].latx, np.average( scale_field*Al[2].AA_s_yx, axis=1 ) )


In [ ]:
#plt.plot( np.average( scale_field*Al[0].AA_s_yx , axis=1 ) )
plt.plot( Al[0].latx, np.average( scale_field*(Al[1].AA_s_yx -Al[0].AA_s_yx) , axis=1 ) )
plt.plot( Al[0].latx, np.average( scale_field*(Al[2].AA_s_yx -Al[0].AA_s_yx), axis=1 ) )


In [ ]:
#####